# Building a Crimes Database with Postgres

The purpose of this project is to create a database for storing data related to crimes that occurred in Boston. 

The objectives for this database are:

- Create a database ```crimes_db``` with a table ```boston_crimes``` with appropriate data types
- Create a schema and create the table inside it
- Loading the data from ```boston.csv``` in the table
- Create readonly and readwrite groups with appropriate privileges
- Create users for each of the groups

In [2]:
import psycopg2
conn = psycopg2.connect(host = "rds-postgres-crimes.c2vaexwxf4rt.us-east-2.rds.amazonaws.com", database = "postgres",
user = "postgres", password= "#####", port=5432)

In [3]:
# import psycopg2
# conn = psycopg2.connect("dbname = dq user = dq")
cur = conn.cursor()
conn.autocommit = True
cur.execute('CREATE DATABASE crimes_db;')
conn.close()

In [5]:
conn = psycopg2.connect(host = "rds-postgres-crimes.c2vaexwxf4rt.us-east-2.rds.amazonaws.com", database = "crimes_db",
user = "postgres", password= "postgres123", port=5432)
cur = conn.cursor()
cur.execute('CREATE SCHEMA crimes_sch;')

In [7]:
import csv
rows = []
with open('boston.csv') as file:
    reader = csv.reader(file)
    for row in reader:
        rows.append(row)
        
col_headers = rows[0]
first_row = row[1]

print(rows[0])
print(rows[1])

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [8]:
import pandas as pd
def get_col_set(file, index):
    df = pd.read_csv(file)
    return set(df.iloc[:,index])

In [9]:
for item in range(len(col_headers)):
    print(len(get_col_set("boston.csv", item)))

298329
219
239
1177
7
18177
18177


In [10]:
col_values = get_col_set("boston.csv", 2)
len(max(col_values, key=len))

58

In [11]:
off_code_col = get_col_set("boston.csv", 1)
print(min(off_code_col), max(off_code_col))

111 3831


In [12]:
lat_col = get_col_set("boston.csv", -2)
print(min(lat_col), max(lat_col))

42.2324133 42.39504158


In [13]:
long_col = get_col_set("boston.csv", -1)
print(min(long_col), max(long_col))

-71.17867378 -70.96367615


After inspecting the dataset content, I can move on to creating the table. To do that, the datatypes must be specified for each column. 

Before I create the table, I create an enumerated datatype for the day of the week column:

In [14]:
cur.execute("CREATE TYPE days_week AS ENUM ('Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday');")

Now, for the creation of the table:

In [15]:
cur.execute("""CREATE TABLE crimes_sch.boston_crimes (
    incident_number serial PRIMARY KEY,
    offense_code smallint,
    description VARCHAR(100),
    date date,
    day_of_the_week days_week,
    lat decimal(10,8),
    long decimal(10,8)
);""")

In [16]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes_sch.boston_crimes FROM STDIN WITH CSV HEADER;", f)

Check that all rows have been copied over:

In [17]:
cur.execute("SELECT * FROM crimes_sch.boston_crimes;")
print(len(cur.fetchall()))

298329


Revoke public users privileges:

In [18]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crimes_db FROM public;")
conn.commit()

Creating readonly and readwrite groups with appropriate privileges and assigning data analysts and scientists to them:

In [19]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")

cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readonly;")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readwrite;")

cur.execute("GRANT USAGE ON SCHEMA crimes_sch TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes_sch TO readwrite;")

cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes_sch TO readonly;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes_sch TO readwrite;")

conn.commit()

In [20]:
cur.execute("""CREATE USER data_analyst WITH PASSWORD 'secret1';""")
cur.execute("""GRANT data_analyst TO readonly;""")
cur.execute("""CREATE USER data_scientist WITH PASSWORD 'secret2';""")
cur.execute("""GRANT data_scientist TO readwrite;""")

conn.commit()

In [21]:
conn.close()